In [ ]:
!pip install deepface
!pip install opencv-python
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install matplotlib
!pip install yolov5 --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.8 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.8 MB/s 

In [ ]:
from deepface import DeepFace
import cv2
import numpy as np

# Criminal images
criminal_db = {
    "Nzubechukwu Dominic Nwoke": "me.jpg",
    # Add more criminals here
}

print("Embedding criminals...")
criminal_embeddings = {}

for name, path in criminal_db.items():
    faces = DeepFace.extract_faces(
        img_path=path,
        detector_backend="mtcnn",
        enforce_detection=False,
        align=False
    )
    if len(faces) == 0:
        print(f"No face found for {name}")
        continue

    fa = faces[0]["facial_area"]
    img = cv2.imread(path)
    x, y, w, h = fa["x"], fa["y"], fa["w"], fa["h"]
    crop = img[y:y+h, x:x+w]

    aligned = DeepFace.represent(
        img_path=crop,
        model_name="ArcFace",
        detector_backend="retinaface",
        enforce_detection=False
    )

    cr_emb = np.array(aligned[0]["embedding"])
    criminal_embeddings[name] = cr_emb
    print(f"Embedded: {name}")

print("\nCriminal DB Ready\n")

Embedding criminals...
Embedded: Nzubechukwu Dominic Nwoke

Criminal DB Ready



In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np

def get_webcam_frame():
    js = Javascript('''
    async function captureFrame() {
      const video = document.createElement('video');
      video.width = 640;
      video.height = 480;
      video.autoplay = true;
      document.body.appendChild(video);
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;
      await video.play();

      // Capture a single frame after 1 second
      await new Promise(resolve => setTimeout(resolve, 1000));

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      const dataUrl = canvas.toDataURL('image/jpeg', 0.8);

      stream.getTracks().forEach(track => track.stop());
      video.remove();
      return dataUrl;
    }
    ''')
    display(js)
    data = eval_js("captureFrame()")
    binary = b64decode(data.split(',')[1])
    img = cv2.imdecode(np.frombuffer(binary, np.uint8), cv2.IMREAD_COLOR)
    return img

In [ ]:
import torch

# Load YOLOv5
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
print("YOLOv5 loaded")

CONF_THRESHOLD = 0.5
person_detected = False
capture_filename = "detected_person.jpg"

print("Monitoring webcam for people...")

while not person_detected:
    frame = get_webcam_frame()
    results = yolo_model(frame)
    labels, cords = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

    for i, label in enumerate(labels):
        if int(label) == 0:  # 0 = person
            conf = cords[i][4].item()
            if conf > CONF_THRESHOLD:
                person_detected = True
                cv2.imwrite(capture_filename, frame)
                print(f"Person detected! Image saved as {capture_filename}")
                break

print("Webcam monitoring stopped. Proceeding to face recognition...")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-11-15 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLOv5 loaded
Monitoring webcam for people...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Person detected! Image saved as detected_person.jpg
Webcam monitoring stopped. Proceeding to face recognition...


In [ ]:
from deepface import DeepFace
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# -----------------------------
# Step 0: Criminal database
# -----------------------------
# Example format: {"name": "path_to_image.jpg"}
# -----------------------------
# Step 1: Load snapshot
# -----------------------------
# -----------------------------
# Step 2: Detect Faces (MTCNN)
# -----------------------------
img_path = "choirs.jpg"

faces = DeepFace.extract_faces(
    img_path=img_path,
    detector_backend="mtcnn",
    enforce_detection=False,
    align=False
)

print(f"Detected {len(faces)} face(s)\n")

img_display = cv2.imread(img_path)


# -----------------------------
# Step 3: Face-by-face analysis (RetinaFace for alignment)
# -----------------------------
for i, face_data in enumerate(faces, 1):
    print(f"Processing face {i}...")

    fa = face_data["facial_area"]
    x, y, w, h = fa["x"], fa["y"], fa["w"], fa["h"]

    # Crop face region
    crop = img_display[y:y+h, x:x+w]

    # Align with RetinaFace before embedding
    aligned = DeepFace.represent(
        img_path=crop,
        detector_backend="retinaface",
        model_name="ArcFace",
        enforce_detection=False
    )

    emb = np.array(aligned[0]["embedding"])

    best_name = "UNKNOWN"
    best_similarity = -1

    # Compare with DB
    for name, c_emb in criminal_embeddings.items():

        sim = np.dot(emb, c_emb) / (np.linalg.norm(emb) * np.linalg.norm(c_emb))
        print(f"   Similarity with {name}: {sim:.4f}")

        if sim > best_similarity:
            best_similarity = sim
            best_name = name

    accuracy = round(best_similarity * 100, 1)

    label = best_name if best_similarity > 0.5 else "UNKNOWN"

    print(f" -> Final decision for face {i}: {label} {accuracy}%\n")

    # Draw box + label
    color = (0, 0, 255) if label != "UNKNOWN" else (0, 255, 0)
    cv2.rectangle(img_display, (x, y), (x + w, y + h), color, 2)
    cv2.putText(
        img_display,
        f"{label} {accuracy}%",
        (x, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        color,
        2
    )


# -----------------------------
# Step 4: Show Output
# -----------------------------
from google.colab.patches import cv2_imshow
cv2_imshow(img_display)
cv2.imwrite("output_with_boxes.jpg", img_display)
print("Saved as output_with_boxes.jpg")

Output hidden; open in https://colab.research.google.com to view.